# **Project Title-** Local Food Wastage Management System

## **Problem Statement**
Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:
- Restaurants and individuals can list surplus food.
- NGOs or individuals in need can claim the food.
- SQL stores available food details and locations.
- A Streamlit app enables interaction, filtering, CRUD operation, and visualization. 


In [1]:
# Import Necessary libraires
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

In [2]:
providers_df = pd.read_csv("providers_data.csv")
receivers_df = pd.read_csv("receivers_data.csv")
food_listing_df = pd.read_csv("food_listings_data.csv")
claims_df = pd.read_csv("claims_data.csv")
print("All the datasets are successfully loaded!!!")

All the datasets are successfully loaded!!!


In [3]:
providers_df.head()

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


In [4]:
receivers_df.head()

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154


In [5]:
food_listing_df.head()

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner


In [6]:
claims_df.head()

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17


### check consistency in data

In [7]:
# check null values if any in providers dataframe. 
print("NULL VALUES IN ANY DATAFRAME: -\n")
print(providers_df.isna().isnull().sum())
print("----------------------------------")

# check null values if any in receivers dataframe.
print(receivers_df.isna().isnull().sum())
print("----------------------------------")

# check null values if any in food_listing dataframe.
print(food_listing_df.isna().isnull().sum())
print("----------------------------------")

# check null values if any in claims dataframe.
print(claims_df.isna().isnull().sum())
print("----------------------------------")

NULL VALUES IN ANY DATAFRAME: -

Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64
----------------------------------
Receiver_ID    0
Name           0
Type           0
City           0
Contact        0
dtype: int64
----------------------------------
Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64
----------------------------------
Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64
----------------------------------


In [8]:
#check information summary of dataFrames
print("Information Summary:\n")
print(providers_df.info())
print("-----------------------------------------")

print(receivers_df.info())
print("-----------------------------------------")

print(food_listing_df.info())
print("-----------------------------------------")

print(claims_df.info())
print("-----------------------------------------")

Information Summary:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Provider_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   Address      1000 non-null   object
 4   City         1000 non-null   object
 5   Contact      1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB
None
-----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Receiver_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   City         1000 non-null   object
 4   Contact      1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
None

In [59]:
# convert date column into datetime format in food_listing columns
print(food_listing_df['Expiry_Date'].dtype)

object


In [3]:
food_listing_df['Expiry_Date'] = pd.to_datetime(food_listing_df['Expiry_Date'], format="%m/%d/%Y").dt.date
print(food_listing_df['Expiry_Date'])

0      2025-03-17
1      2025-03-24
2      2025-03-28
3      2025-03-16
4      2025-03-19
          ...    
995    2025-03-30
996    2025-03-18
997    2025-03-22
998    2025-03-30
999    2025-03-19
Name: Expiry_Date, Length: 1000, dtype: object


## Note
### To convert the string "3/5/2025 5:26" into a date object in Python, utilize the datetime module and specifically the strptime() method.
**The following steps demonstrate this conversion:**
- Import the **datetime class**: This class is part of the datetime module and provides functionalities for handling dates and times.
- Define the date string and its format: The strptime() method requires both the string to be converted and a format string that matches the structure of the input.
- **%m**: Month as a zero-padded decimal number (e.g., 03 for March).
- **%d**: Day of the month as a zero-padded decimal number (e.g., 05).
- **%Y**: Year with century as a decimal number (e.g., 2025).
- **%H**: Hour (24-hour clock) as a zero-padded decimal number (e.g., 05).
- **%M**: Minute as a zero-padded decimal number (e.g., 26).

- **Convert the string to a datetime object:** Use datetime.strptime() to perform the conversion.
- **Extract the date part:** If only the date is required, use the .date() method on the datetime object.

In [4]:
# Similarly, convert the TimeStamp Column into date time format
claims_df['Timestamp'] = pd.to_datetime(claims_df['Timestamp'], format="%m/%d/%Y %H:%M")
print(claims_df['Timestamp'])

0     2025-03-05 05:26:00
1     2025-03-11 10:24:00
2     2025-03-21 00:59:00
3     2025-03-04 09:08:00
4     2025-03-14 15:17:00
              ...        
995   2025-03-13 19:40:00
996   2025-03-17 22:04:00
997   2025-03-13 18:00:00
998   2025-03-01 15:31:00
999   2025-03-13 12:51:00
Name: Timestamp, Length: 1000, dtype: datetime64[ns]


When you use **pd.to_datetime**, the resulting Series (or column in this case) will already be of the datetime64 dtype, which is what you want for working with dates and times in pandas. 
The **.dt** accessor in pandas is used to access datetime properties of a Series, and then you can access attributes like .dt.date (to get just the date component) or .dt.time (to get just the time component). However, .dt.datetime isn't a valid attribute or method itself to call. You're essentially trying to call an attribute as if it were a function, leading to a TypeError. 
Corrected Code
To fix this, simply remove **.dt.datetime** from the end of the line:

### In Python's datetime module, dt.date and dt.datetime represent different levels of temporal precision:
- **dt.date:** This class represents a calendar date (year, month, and day) without any time-of-day information. It is suitable when you only need to store or manipulate the date component of a timestamp and do not require details about hours, minutes, seconds, or microseconds.
- **dt.datetime:** This class represents a specific moment in time, combining both date and time information down to the microsecond. It includes year, month, day, hour, minute, second, and microsecond. It is used when you need precise timestamps that capture both the date and the exact time.

In [5]:
import mysql.connector
# DataBase Connection
 
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = '12345'
)
# MySQL connection Setup
cursor = conn.cursor()

In [5]:
#!pip install mysql-connector-python

In [33]:
cursor.execute("CREATE DATABASE IF NOT EXISTS food_data")
print("MYSQL DataBase 'food_data' created successfully!!!")

MYSQL DataBase 'food_data' created successfully!!!


In [34]:
cursor.execute("USE food_data")

In [35]:
providers_df.columns

Index(['Provider_ID', 'Name', 'Type', 'Address', 'City', 'Contact'], dtype='object')

In [36]:
len(providers_df.iloc[3][3])

C:\Users\uttam\AppData\Local\Temp\ipykernel_29744\3784070299.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  len(providers_df.iloc[3][3])


45

In [37]:
cursor.execute("""
     CREATE TABLE IF NOT EXISTS providers (
     Provider_ID INT PRIMARY KEY,
     Name VARCHAR(255),
     Type VARCHAR(255),
     Address TEXT,
     City VARCHAR(100),
     Contact VARCHAR(100)
     )
""")
conn.commit()

In [38]:
# Insert providers Dataframe data into the MYSQL database
for index, row in providers_df.iterrows():
    cursor.execute("""
    INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact)
    VALUES (%s, %s, %s, %s, %s, %s)
    """,tuple(row))    

conn.commit()  # Commit the INSERT statements after the loop completes
print(f"{cursor.rowcount} records inserted successfully.")

1 records inserted successfully.


### Why %s is generally preferred?
Many database connectors, including psycopg2 for PostgreSQL and the sqlite3 module for SQLite, as well as MySQL Connector/Python, treat the %s placeholder as a generic placeholder for values. The connector then handles the appropriate type conversion from the Python object to the correct SQL data type, including integers, decimals, strings, and dates.

In [31]:
# SQL query to get the city with the highest number of providers
query = """
    SELECT City, COUNT(*) AS Provider_Count
    FROM providers
    GROUP BY City
    ORDER BY Provider_Count DESC
    LIMIT 1;
"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["City", "Provider_Count"])
df

,City,Provider_Count
0,New Carol,3


In [46]:
# create table receivers in food_data database 
cursor.execute("""
     CREATE TABLE IF NOT EXISTS receivers (
     Receivers_ID INT PRIMARY KEY,
     Name VARCHAR(255),
     Type VARCHAR(255),
     City VARCHAR(100),
     Contact VARCHAR(100)
     )
""")
conn.commit()

In [47]:
receivers_df.columns

Index(['Receiver_ID', 'Name', 'Type', 'City', 'Contact'], dtype='object')

In [48]:
# Insert data into the receivers table.
for index, row in receivers_df.iterrows():
    cursor.execute("""
    INSERT INTO RECEIVERS (Receivers_ID, Name, Type, City, Contact)
    VALUES (%s, %s, %s, %s, %s)
    """,tuple(row))
conn.commit()
print(f'{cursor.rowcount} rows inserted rows successfully.')    

1 rows inserted rows successfully.


In [56]:
# create table food_listings in food_data database 
cursor.execute("""
     CREATE TABLE IF NOT EXISTS food_listings (
     Food_ID INT PRIMARY KEY,
     Food_Name VARCHAR(255),
     Quantity INT,
     Expiry_Date DATE,
     Provider_ID INT,
     Provider_Type VARCHAR(100),
     Location VARCHAR(100),
     Food_Type VARCHAR(100),
     Meal_Type VARCHAR(100)
     )
""")
conn.commit()

In [49]:
food_listing_df.columns

Index(['Food_ID', 'Food_Name', 'Quantity', 'Expiry_Date', 'Provider_ID',
       'Provider_Type', 'Location', 'Food_Type', 'Meal_Type'],
      dtype='object')

In [67]:
# Insert data into food_listings tables
for index, row in food_listing_df.iterrows():
    cursor.execute("""
    INSERT INTO food_listings(Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID,
                              Provider_Type, Location, Food_Type, Meal_Type)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)   
    """, tuple(row))

conn.commit()
print(f'{cursor.rowcount} tabel updated.' )

1 tabel updated.


In [72]:
# create table claims in food_data database 
cursor.execute("""
     CREATE TABLE IF NOT EXISTS claims (
     Claim_ID INT PRIMARY KEY,
     Food_ID INT,
     Receiver_ID INT,
     Status VARCHAR(100),
     Timestamp DATETIME
     )
""")
conn.commit()

In [73]:
claims_df.columns

Index(['Claim_ID', 'Food_ID', 'Receiver_ID', 'Status', 'Timestamp'], dtype='object')

In [74]:
# Insert data into claims tables
for index, row in claims_df.iterrows():
    cursor.execute("""
    INSERT INTO claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
    VALUES(%s, %s, %s, %s, %s)   
    """, tuple(row))

conn.commit()
print(f'{cursor.rowcount} tabel updated.' )

1 tabel updated.


# All data tables entries are completed in DATABASE of MYSQL now.

MySQL does not directly support the FULL OUTER JOIN keyword like some other SQL databases. However, you can achieve the same result by combining LEFT JOIN and RIGHT JOIN with the UNION operator. 

When performing a simulated FULL OUTER JOIN in MySQL, especially when using SELECT * from both tables, you might encounter issues if both tables contain a column with the same name (e.g., Name in both Customers and Employees). This would lead to a "duplicate column name" error.

1. Use table aliases and explicit column selection
Instead of using SELECT *, specify the columns you want to retrieve and use table aliases to distinguish between columns with the same name.
2. USING clause (when join columns have identical names)
If the column you're joining on has the same name in both tables (e.g., id in both Customers and Orders tables for a customer_id relationship), you can use the USING clause. The USING clause automatically handles the duplicate column by displaying it only once in the result set. However, you'll still need to alias other conflicting columns like Name.